# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
collection_names = db.list_collection_names()
print("Collections in the database:", collection_names)

Collections in the database: ['establishments']


In [9]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [11]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print("Number of establishments with a hygiene score equal to 20:", hygiene_20_count)
# Display the first document in the results using pprint
hygiene_20_doc = establishments.find_one(query)
pprint(hygiene_20_doc)

Number of establishments with a hygiene score equal to 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64d2f437e9489ea9f776826d'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 

In [12]:
import pandas as pd

# Convert the result to a Pandas DataFrame
hygiene_20_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
num_rows = len(hygiene_20_df)
print("Number of rows in the DataFrame:", num_rows)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(hygiene_20_df.head(10))


Number of rows in the DataFrame: 41
First 10 rows of the DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  64d2f437e9489ea9f776826d   110681                  0   
1  64d2f437e9489ea9f77685eb   612039                  0   
2  64d2f437e9489ea9f7768903   730933                  0   
3  64d2f437e9489ea9f7768ae4   172735                  0   
4  64d2f437e9489ea9f7768af6   172953                  0   
5  64d2f437e9489ea9f776949d   512854                  0   
6  64d2f437e9489ea9f77696b5  1537089                  0   
7  64d2f437e9489ea9f776abdc   155648                  0   
8  64d2f437e9489ea9f776b02d  1012883                  0   
9  64d2f438e9489ea9f776b838   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza 

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": ".*London.*"},
    "RatingValue": {"$gte": 4}
}
# Use count_documents to display the number of documents in the result
london_high_rating_count = establishments.count_documents(query)
print("Number of establishments in London with a RatingValue greater than or equal to 4:", london_high_rating_count)
# Display the first document in the results using pprint
london_high_rating_doc = establishments.find_one(query)
pprint(london_high_rating_doc)

In [13]:
# Convert the result to a Pandas DataFrame
london_high_rating_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
num_rows_london = len(london_high_rating_df)
print("Number of rows in the London DataFrame:", num_rows_london)
# Display the first 10 rows of the DataFrame
print("First 10 rows of the London DataFrame:")
print(london_high_rating_df.head(10))

Number of rows in the London DataFrame: 41
First 10 rows of the London DataFrame:
                        _id   FHRSID  ChangesByServerID  \
0  64d2f437e9489ea9f776826d   110681                  0   
1  64d2f437e9489ea9f77685eb   612039                  0   
2  64d2f437e9489ea9f7768903   730933                  0   
3  64d2f437e9489ea9f7768ae4   172735                  0   
4  64d2f437e9489ea9f7768af6   172953                  0   
5  64d2f437e9489ea9f776949d   512854                  0   
6  64d2f437e9489ea9f77696b5  1537089                  0   
7  64d2f437e9489ea9f776abdc   155648                  0   
8  64d2f437e9489ea9f776b02d  1012883                  0   
9  64d2f438e9489ea9f776b838   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [14]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

latitude_range = (latitude - degree_search, latitude + degree_search)
longitude_range = (longitude - degree_search, longitude + degree_search)

query = query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude_range[0], "$lte": latitude_range[1]},
    "geocode.longitude": {"$gte": longitude_range[0], "$lte": longitude_range[1]}
}
sort = [("scores.Hygiene", 1)]

results = establishments.find(query).sort(sort).limit(5)

# Print the results
print("Top 5 establishments with RatingValue 5, sorted by lowest hygiene score and nearest to Penang Flavours:")
for idx, result in enumerate(results, 1):
    print(f"Result {idx}:")
    pprint(result)
    print()

Top 5 establishments with RatingValue 5, sorted by lowest hygiene score and nearest to Penang Flavours:
Result 1:
{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'BusinessType': 'Mobile caterer',
 'BusinessTypeID': 7846,
 'ChangesByServerID': 0,
 'Distance': 4646.95593107927,
 'FHRSID': 1380578,
 'LocalAuthorityBusinessID': '14425',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1SE',
 'RatingDate': '2021-11-11T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64d2f438e9489ea9f776daad'),
 'geocode': {'latitude': 51.4875335693359, 'longitude': 0.09253700077

In [15]:
# Convert result to Pandas DataFrame
# Create an empty list to store results
top_rating_5_results = []

# Iterate through the results and append them to the list
for result in results:
    top_rating_5_results.append(result)

# Convert the list of results to a Pandas DataFrame
top_rating_5_df = pd.DataFrame(top_rating_5_results)

# Display the Pandas DataFrame
print("Top 5 establishments with RatingValue 5, sorted by lowest hygiene score and nearest to Penang Flavours:")
print(top_rating_5_df)

Top 5 establishments with RatingValue 5, sorted by lowest hygiene score and nearest to Penang Flavours:
Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [16]:
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline
hygiene_0_by_local_authority = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents with hygiene score 0 by Local Authority:")
print(len(hygiene_0_by_local_authority))

# Print the first 10 results
print("Top 10 Local Authority areas with the most establishments having hygiene score 0:")
for idx, result in enumerate(hygiene_0_by_local_authority[:10], 1):
    print(f"{idx}. {result['_id']}: {result['count']} establishments")

Number of documents with hygiene score 0 by Local Authority:
55
Top 10 Local Authority areas with the most establishments having hygiene score 0:
1. Thanet: 1130 establishments
2. Greenwich: 882 establishments
3. Maidstone: 713 establishments
4. Newham: 711 establishments
5. Swale: 686 establishments
6. Chelmsford: 680 establishments
7. Medway: 672 establishments
8. Bexley: 607 establishments
9. Southend-On-Sea: 586 establishments
10. Tendring: 542 establishments


In [17]:
# Convert the result to a Pandas DataFrame
hygiene_0_df = pd.DataFrame(hygiene_0_by_local_authority)

# Display the number of rows in the DataFrame
num_rows_hygiene_0 = len(hygiene_0_df)
print("Number of rows in the DataFrame:", num_rows_hygiene_0)

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(hygiene_0_df.head(10))

Number of rows in the DataFrame: 55
First 10 rows of the DataFrame:
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
